In [1]:
import json
import numpy as np
import cv2
from glob import glob
from os.path import join as pjoin
from tqdm import tqdm

In [33]:
def load_result_multi_json(reslut_file_root):
    def is_bottom_or_top(corner):
        column_min, row_min, column_max, row_max = corner
        if row_max < 36 or row_min > 725:
            return True
        return False

    result_files = glob(pjoin(reslut_file_root, '*.json'))
    compos_reform = {}
    print('Loading %d detection results' % len(result_files))
    for reslut_file in tqdm(result_files):
        img_name = reslut_file.split('\\')[-1].split('.')[0]
        compos = json.load(open(reslut_file, 'r'))['compos']
        for compo in compos:
            if is_bottom_or_top((compo['column_min'], compo['row_min'], compo['column_min'] + compo['column_max'], compo['row_min'] + compo['row_max'])):
                continue
            if img_name not in compos_reform:
                compos_reform[img_name] = {'bboxes': [[compo['column_min'], compo['row_min'], compo['column_min'] + compo['column_max'], compo['row_min'] + compo['row_max']]],
                                           'categories': [compo['class']]}
            else:
                compos_reform[img_name]['bboxes'].append([compo['column_min'], compo['row_min'], compo['column_min'] + compo['column_max'], compo['row_min'] + compo['row_max']])
                compos_reform[img_name]['categories'].append(compo['class'])
    return compos_reform

In [34]:
detect = load_result_multi_json('E:\\Mulong\\Result\\ocr')

  0%|                                                                                 | 7/5066 [00:00<01:21, 62.01it/s]

Loading 5066 detection results


100%|█████████████████████████████████████████████████████████████████████████████| 5066/5066 [00:49<00:00, 101.46it/s]


In [36]:
def merge_text(corners, org_shape, max_word_gad=10, max_word_height_ratio=0.05):
    def is_text_line(corner_a, corner_b):
        ((col_min_a, row_min_a), (col_max_a, row_max_a)) = corner_a
        ((col_min_b, row_min_b), (col_max_b, row_max_b)) = corner_b
        # on the same line
        if abs(row_min_a - row_min_b) < max_word_gad and abs(row_max_a - row_max_b) < max_word_gad:
            # close distance
            if abs(col_min_b - col_max_a) < max_word_gad or abs(col_min_a - col_max_b) < max_word_gad:
                return True
        return False

    def corner_merge_two_corners(corner_a, corner_b):
        ((col_min_a, row_min_a), (col_max_a, row_max_a)) = corner_a
        ((col_min_b, row_min_b), (col_max_b, row_max_b)) = corner_b

        col_min = min(col_min_a, col_min_b)
        col_max = max(col_max_a, col_max_b)
        row_min = min(row_min_a, row_min_b)
        row_max = max(row_max_a, row_max_b)
        return (col_min, row_min), (col_max, row_max)

    changed = False
    new_corners = []
    row, col = org_shape[:2]
    for i in range(len(corners)):
        merged = False
        height = corners[i][1][1] - corners[i][0][1]
        # ignore non-text
        if height / row > max_word_height_ratio:
            new_corners.append(corners[i])
            continue
        for j in range(len(new_corners)):
            if is_text_line(corners[i], new_corners[j]):
                new_corners[j] = corner_merge_two_corners(corners[i], new_corners[j])
                merged = True
                changed = True
                break
        if not merged:
            new_corners.append(corners[i])

    if not changed:
        return corners
    else:
        return merge_text(new_corners, org_shape)


In [40]:
for img_id in detect:
    img = cv2.imread('E:\\Mulong\\Datasets\\rico\\combined' + '\\' + img_id + '.jpg')
    merge_text(detect[img_id]['bboxes'], img.shape)

TypeError: 'int' object is not subscriptable